In [ ]:
from gliopath.models.load import giga_slide_enc

import torch
from tqdm.notebook import tqdm
from glob import glob
import os

os.chdir('F:/workspace/pathology/gigapath')
slide_encoder = giga_slide_enc(path='model/pub/slide_encoder.pth', global_pool=True)

## use trained model to infer

In [ ]:
slide_pt_paths = glob('output/tiles/*.pt', recursive=False)
slide_collated = {}

for slide_pt_path in tqdm(slide_pt_paths):
    slide_pt = torch.load(slide_pt_path)
    tile_embeds = slide_pt['tile_embeds']
    coords = slide_pt['coords']
    
    if len(tile_embeds.shape) == 2:
        tile_embeds = tile_embeds.unsqueeze(0)
        coords = coords.unsqueeze(0)
    
    slide_encoder = slide_encoder.cuda()
    slide_encoder.eval()
    
    # run inference
    with torch.cuda.amp.autocast(dtype=torch.float16):
        slide_embeds = slide_encoder(tile_embeds.cuda(), coords.cuda(), all_layer_embed=True)
    outputs = {"layer_{}_embed".format(i): slide_embeds[i].cpu() for i in range(len(slide_embeds))}
    outputs["last_layer_embed"] = slide_embeds[-1].cpu()

    slide_nom = slide_pt_path.split('/')[-1].split('.')[0]
    slide_collated[slide_nom] = outputs

In [ ]:
torch.save(slide_collated, 'output/all_slides_embeds.pt')

## fine tuning this model